In [1]:
# Code from 
import torch
import math

dtype = torch.float
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-math.pi, math.pi, 2000, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For a third order polynomial, we need
# 4 weights: y = a + b x + c x^2 + d x^3
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.randn((), dtype=dtype, requires_grad=True)
b = torch.randn((), dtype=dtype, requires_grad=True)
c = torch.randn((), dtype=dtype, requires_grad=True)
d = torch.randn((), dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y using operations on Tensors.
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
    # the gradient of the loss with respect to a, b, c, d respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

/home/jessica/miniconda3/envs/torch-test/lib/python3.12/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


99 972.1571655273438
199 651.276611328125
299 437.50341796875
399 295.0176086425781
499 200.00001525878906
599 136.6045379638672
699 94.28449249267578
799 66.01742553710938
899 47.12554168701172
999 34.49162292480469
1099 26.037155151367188
1199 20.375654220581055
1299 16.581735610961914
1399 14.0374755859375
1499 12.32996940612793
1599 11.183087348937988
1699 10.412118911743164
1799 9.893401145935059
1899 9.54409408569336
1999 9.308650016784668
Result: y = 0.013585366308689117 + 0.8391690254211426 x + -0.002343702595680952 x^2 + -0.09083095192909241 x^3


In [15]:
import pandas as pd
import numpy as np

df = pd.read_csv("stock-data.csv")
df.info()
df.drop("Date", inplace=True, axis = 1)
df = df.replace(",","", regex = True)
df = df.astype(float)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    232 non-null    object
 1   Open    232 non-null    object
 2   High    232 non-null    object
 3   Low     232 non-null    object
 4   Close   232 non-null    object
dtypes: object(5)
memory usage: 9.2+ KB


In [33]:
features = df[["Open", "High", "Low"]]
print(features.info())
features = features.values
target = df[["Close"]].values
# features.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    232 non-null    float64
 1   High    232 non-null    float64
 2   Low     232 non-null    float64
dtypes: float64(3)
memory usage: 5.6 KB
None


In [79]:
# mean = features.mean(axis = 0)
# std = features.std(axis=0)
# features = (features - mean)/std

features = torch.round(torch.tensor(features, dtype = torch.float64)*100/100)

#this is beacuse we want a 1D array for our answerers
target = torch.round(torch.tensor(target, dtype = torch.float64).view(-1,1)*100)/100


/home/jessica/miniconda3/envs/torch-test/lib/python3.12/site-packages/torch/utils/_device.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


In [81]:
import torch.nn as nn
import torch.nn.functional as F

class stockPredictor(nn.Module):
    def __init__(self):
        super(stockPredictor, self).__init__()
        #this data has 4 inputs than we want to go to 64 i dont not understand why we chose these numbers
        self.fc1 = nn.Linear(3,63)
        self.fc2 = nn.Linear(63,32)
        self.fc3 = nn.Linear(32,1)
        self.double()
    
    #goes through each layer of the neural netwrok and returns an answer depending
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = stockPredictor()

In [80]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state = 41)
print(y_test)

tensor([[4536.3400],
        [5234.1800],
        [4754.6300],
        [5307.0100],
        [5070.5500],
        [5071.6300],
        [5204.3400],
        [5178.5100],
        [4845.6500],
        [5243.7700],
        [4719.1900],
        [4585.5900],
        [4890.9700],
        [4411.5900],
        [4643.7000],
        [4193.8000],
        [4508.2400],
        [5199.0600],
        [5308.1500],
        [4522.7900],
        [5104.7600],
        [4997.9100],
        [4186.7700],
        [4373.6300],
        [5149.4200],
        [4247.6800],
        [4465.4800],
        [4376.3100],
        [5137.0800],
        [5203.5800],
        [4274.5100],
        [5087.0300],
        [4468.8300],
        [4505.4200],
        [5096.2700],
        [4780.9400],
        [4314.6000],
        [4472.1600],
        [4569.7800],
        [5308.1300],
        [4451.1400],
        [4769.8300],
        [4768.3700],
        [4507.6600],
        [5035.6900],
        [4478.0300],
        [4399.7700]], dtype=torch.

In [84]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)

epochs = 500
losses = []
for epoch in range(epochs):

    y_pred = model.forward(x_train)

    outputs = model(features)
    loss = criterion(y_pred, y_train)

    losses.append(loss.detach().numpy())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch +1) %10 ==0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss}')

Epoch [10/500], Loss: 111519.7828668478
Epoch [20/500], Loss: 25215.477989263854
Epoch [30/500], Loss: 3393.6843144882705
Epoch [40/500], Loss: 649.8443699380229
Epoch [50/500], Loss: 2183.4645862531365
Epoch [60/500], Loss: 308.53114472245045
Epoch [70/500], Loss: 533.5566027708862
Epoch [80/500], Loss: 304.84162114806094
Epoch [90/500], Loss: 270.7419904949941
Epoch [100/500], Loss: 272.8051579048661
Epoch [110/500], Loss: 271.89562070503297
Epoch [120/500], Loss: 270.89503798070524
Epoch [130/500], Loss: 270.55972638699893
Epoch [140/500], Loss: 270.5529601175776
Epoch [150/500], Loss: 270.5309238044288
Epoch [160/500], Loss: 270.43254453064094
Epoch [170/500], Loss: 270.36780390241296
Epoch [180/500], Loss: 270.3194470295251
Epoch [190/500], Loss: 270.26363679601116
Epoch [200/500], Loss: 270.21000010242335
Epoch [210/500], Loss: 270.15411970331843
Epoch [220/500], Loss: 270.09707631587577
Epoch [230/500], Loss: 270.038625370523
Epoch [240/500], Loss: 269.978724583465
Epoch [250/50

In [85]:
with torch.no_grad():
    y_eval = model.forward(x_test)
    loss = criterion(y_eval,y_test)

In [86]:
num_correct = 0
with torch.no_grad():
    for i, data in enumerate(x_test):
        y_val = model.forward(data)
        print(f"Predicted: {round(y_val.item(),2)} Expected: {round(y_test[i].item(),2)}")
        if round(y_val.item(),2) == round(y_test[i].item(),2):
            num_correct += 1
print(f"Correct: {num_correct}")

Predicted: 4547.17 Expected: 4536.34
Predicted: 5239.75 Expected: 5234.18
Predicted: 4756.61 Expected: 4754.63
Predicted: 5306.89 Expected: 5307.01
Predicted: 5052.82 Expected: 5070.55
Predicted: 5070.49 Expected: 5071.63
Predicted: 5190.16 Expected: 5204.34
Predicted: 5157.09 Expected: 5178.51
Predicted: 4878.88 Expected: 4845.65
Predicted: 5248.73 Expected: 5243.77
Predicted: 4716.96 Expected: 4719.19
Predicted: 4579.15 Expected: 4585.59
Predicted: 4895.38 Expected: 4890.97
Predicted: 4406.42 Expected: 4411.59
Predicted: 4627.28 Expected: 4643.7
Predicted: 4175.95 Expected: 4193.8
Predicted: 4501.38 Expected: 4508.24
Predicted: 5177.43 Expected: 5199.06
Predicted: 5288.3 Expected: 5308.15
Predicted: 4520.11 Expected: 4522.79
Predicted: 5111.63 Expected: 5104.76
Predicted: 4995.08 Expected: 4997.91
Predicted: 4209.27 Expected: 4186.77
Predicted: 4363.23 Expected: 4373.63
Predicted: 5161.95 Expected: 5149.42
Predicted: 4240.46 Expected: 4247.68
Predicted: 4468.73 Expected: 4465.48
Pred

In [87]:
correct = 0
with torch.no_grad():
    for i, data in enumerate(x_train):
        y_val = model.forward(data)
        print(f"Predicted: {round(y_val.item(),2)} Expected: {round(y_train[i].item(),2)}")
        if round(y_val.item(),2) == round(y_train[i].item(),2):
            correct += 1

Predicted: 4557.86 Expected: 4556.62
Predicted: 5121.87 Expected: 5127.79
Predicted: 4363.18 Expected: 4358.24
Predicted: 4366.84 Expected: 4373.2
Predicted: 4774.57 Expected: 4783.45
Predicted: 4248.64 Expected: 4258.19
Predicted: 4428.94 Expected: 4433.31
Predicted: 5210.24 Expected: 5202.39
Predicted: 4740.39 Expected: 4742.83
Predicted: 4555.15 Expected: 4567.8
Predicted: 4570.95 Expected: 4549.34
Predicted: 4531.15 Expected: 4513.39
Predicted: 5144.79 Expected: 5123.41
Predicted: 4778.32 Expected: 4781.58
Predicted: 5122.31 Expected: 5117.09
Predicted: 4578.06 Expected: 4582.23
Predicted: 5044.22 Expected: 5064.2
Predicted: 4223.76 Expected: 4217.04
Predicted: 4535.66 Expected: 4538.19
Predicted: 5155.97 Expected: 5123.69
Predicted: 4346.98 Expected: 4327.78
Predicted: 5097.88 Expected: 5088.8
Predicted: 5301.0 Expected: 5306.04
Predicted: 4428.97 Expected: 4404.33
Predicted: 4475.32 Expected: 4450.32
Predicted: 5069.05 Expected: 5069.76
Predicted: 4885.27 Expected: 4894.16
Predic